## Image classification with Convolutional Neural Networks

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!ls ../..

CODE-OF-CONDUCT.md  README.md  environment-cpu.yml  requirements.txt  tests
LICENSE		    courses    environment.yml	    setup.cfg	      tutorials
MANIFEST	    datasets   fastai		    setup.py
MANIFEST.in	    docs       pytest.ini	    storage


In [4]:
!ln -s ../../fastai fastai

In [5]:
!ln -s ../../data data

In [9]:
%pwd

'/notebooks/courses/dl1'

In [9]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [10]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [11]:
PATH = "data/plant-seedlings-classification"

In [12]:
torch.cuda.is_available()

True

In [13]:
torch.backends.cudnn.enabled

True

## First look at pictures

In [ ]:
!pip install kaggle

In [ ]:
!export KAGGLE_USERNAME=vandosant
!export KAGGLE_KEY=74c23977351fb1fe6b916b70ce77d3cd

In [ ]:
!export KAGGLE_USERNAME=vandosant && export KAGGLE_KEY=74c23977351fb1fe6b916b70ce77d3cd && kaggle config

In [ ]:
!export KAGGLE_USERNAME=vandosant && export KAGGLE_KEY=74c23977351fb1fe6b916b70ce77d3cd && kaggle competitions files plant-seedlings-classification

In [ ]:
!export KAGGLE_USERNAME=vandosant && export KAGGLE_KEY=74c23977351fb1fe6b916b70ce77d3cd && kaggle competitions download plant-seedlings-classification -p {PATH}

In [ ]:
!mkdir -p path

In [ ]:
!ls {PATH}

In [ ]:
#!rm -r {PATH}/test {PATH}/train {PATH}/images

In [ ]:
!unzip {PATH}/test.zip -d {PATH}

In [ ]:
!unzip {PATH}/train.zip -d {PATH}

In [ ]:
!ls {PATH}

In [ ]:
classes = !ls {PATH}/train | head
classes

In [ ]:
files = !ls {PATH}/train/{classes[0]} | head
files

In [ ]:
img = PIL.Image.open(f'{PATH}/train/{classes[0]}/{files[1]}'); img

In [ ]:
img.size

## Data pre-processing

In [ ]:
from os import listdir
from os.path import join
train_path = f'{PATH}/train'

In [ ]:
dirs = [d for d in listdir(train_path) if os.path.isdir(join(train_path,d))]
print(dirs)

In [ ]:
train_dict = {d: listdir(join(train_path,d)) for d in dirs}

In [ ]:
train_dict.keys()

In [ ]:
sum(len(v) for v in train_dict.values())

In [ ]:
with open(f'{PATH}/train.csv', 'w') as csv:
    csv.write('img,label\n')
    for d in dirs:
        for f in train_dict[d]: csv.write(f'{f},{d.replace(" ", "_")}\n')

In [ ]:
!head {PATH}/train.csv

In [ ]:
!wc -l {PATH}/train.csv

In [ ]:
img_path = f'{PATH}/images'
os.makedirs(img_path, exist_ok=True)

In [ ]:
!cp {PATH}/train/*/*.png {PATH}/images/

In [ ]:
!ls {PATH}/images | wc -l

## Our first model: quick start

In [ ]:
# Uncomment the below if you need to reset your precomputed activations
# shutil.rmtree(f'{PATH}tmp', ignore_errors=True)

In [9]:
arch=resnet34
bs=64
sz=224
csv_fname = os.path.join(PATH, "train.csv")
train_labels = list(open(csv_fname))
n = len(list(open(csv_fname)))-1
val_idxs = get_cv_idxs(n)
tfms = tfms_from_model(arch, sz)
data = ImageClassifierData.from_csv(PATH, "images", csv_fname, bs, tfms, val_idxs, test_name='test')
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 2)

epoch      trn_loss   val_loss   accuracy   
    0      1.558312   0.91203    0.72      
    1      1.110009   0.659725   0.788421  



[array([0.65972]), 0.7884210515022277]

In [ ]:
lrf=learn.lr_find()
learn.sched.plot()

In [ ]:
learn.fit(0.1, 4, cycle_len=1)

### Data augmentation

In [ ]:
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [ ]:
def get_augs():
    data = ImageClassifierData.from_csv(PATH, "images", csv_fname, bs, tfms, val_idxs)
    x,_ = next(iter(data.aug_dl))
    return data.trn_ds.denorm(x)[1]

In [ ]:
ims = np.stack([get_augs() for i in range(6)])

In [ ]:
plots(ims, rows=2)

In [10]:
data = ImageClassifierData.from_csv(PATH, 'images', csv_fname, bs, tfms, val_idxs, test_name='test')
learn = ConvLearner.pretrained(arch, data, precompute=False)

In [ ]:
learn.fit(0.1, 4, cycle_len=1, cycle_mult=2)

In [11]:
learn.sched.plot_lr()

AttributeError: 'NoneType' object has no attribute 'plot_lr'

In [ ]:
learn.save('224_plants_lastlayer')

In [12]:
learn.load('224_plants_lastlayer')

### Fine-tuning and differential learning rate annealing

In [13]:
learn.unfreeze()

In [14]:
lr=np.array([0.1/9,0.1/3,0.1])

In [15]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   accuracy   
    0      0.847065   0.377687   0.870526  
    1      0.503244   0.301592   0.891579  
    2      0.298078   0.168793   0.94      
    3      0.298      0.737075   0.768421  
    4      0.220583   0.231304   0.915789  
    5      0.134473   0.124293   0.952632  
    6      0.080896   0.102979   0.962105  



[array([0.10298]), 0.962105263785312]

In [16]:
learn.save('224_plants_all')

In [17]:
learn.load('224_plants_all')

In [18]:
(learn.data.test_dl == None)

False

In [19]:
log_preds,y = learn.TTA(is_test=True)

In [20]:
log_preds.shape

(5, 794, 12)

In [21]:
probs = np.exp(log_preds)

In [22]:
probs.shape

(5, 794, 12)

In [23]:
type(probs)

numpy.ndarray

In [24]:
ps = np.exp(probs)

In [25]:
len(np.argmax(ps[0], axis=1))

794

In [26]:
df = pd.DataFrame(np.argmax(ps[0], axis=1))

In [27]:
df.insert(0, 'file', [f[5:] for f in data.test_ds.fnames])

In [28]:
df.columns = ['file', 'species']

In [29]:
class_dict = {i: d for i,d in enumerate(data.classes)}

In [30]:
class_dict

{0: 'Black-grass',
 1: 'Charlock',
 2: 'Cleavers',
 3: 'Common_Chickweed',
 4: 'Common_wheat',
 5: 'Fat_Hen',
 6: 'Loose_Silky-bent',
 7: 'Maize',
 8: 'Scentless_Mayweed',
 9: 'Shepherds_Purse',
 10: 'Small-flowered_Cranesbill',
 11: 'Sugar_beet'}

In [31]:
df.head()

,file,species
0,79e5ea8fa.png,4
1,3a909ead8.png,10
2,599691cd9.png,10
3,5ca2687a4.png,7
4,3dd52bd2a.png,10


In [32]:
df["species"] = df["species"].map(class_dict.get)

In [33]:
df.head()

,file,species
0,79e5ea8fa.png,Common_wheat
1,3a909ead8.png,Small-flowered_Cranesbill
2,599691cd9.png,Small-flowered_Cranesbill
3,5ca2687a4.png,Maize
4,3dd52bd2a.png,Small-flowered_Cranesbill


In [34]:
df["species"] = df["species"].map(lambda s: s.replace("_", " "))

In [35]:
df.head()

,file,species
0,79e5ea8fa.png,Common wheat
1,3a909ead8.png,Small-flowered Cranesbill
2,599691cd9.png,Small-flowered Cranesbill
3,5ca2687a4.png,Maize
4,3dd52bd2a.png,Small-flowered Cranesbill


In [36]:
SUBM = f'{PATH}/subm'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}/subm2.gz', compression='gzip', index=False)

In [37]:
FileLink(f'{SUBM}/subm2.gz')

/notebooks/courses/dl1/data/plant-seedlings-classification/subm/subm2.gz

## Review: easy steps to train a world-class image classifier

1. Enable data augmentation, and precompute=True
1. Use `lr_find()` to find highest learning rate where loss is still clearly improving
1. Train last layer from precomputed activations for 1-2 epochs
1. Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1
1. Unfreeze all layers
1. Set earlier layers to 3x-10x lower learning rate than next higher layer
1. Use `lr_find()` again
1. Train full network with cycle_mult=2 until over-fitting

## Analyzing results: loss and accuracy

When we run `learn.fit` we print 3 performance values (see above.) Here 0.03 is the value of the **loss** in the training set, 0.0226 is the value of the loss in the validation set and 0.9927 is the validation accuracy. What is the loss? What is accuracy? Why not to just show accuracy?

**Accuracy** is the ratio of correct prediction to the total number of predictions.

In machine learning the **loss** function or cost function is representing the price paid for inaccuracy of predictions.

The loss associated with one example in binary classification is given by:
`-(y * log(p) + (1-y) * log (1-p))`
where `y` is the true label of `x` and `p` is the probability predicted by our model that the label is 1.

In [38]:
def binary_loss(y, p):
    return np.mean(-(y * np.log(p) + (1-y)*np.log(1-p)))

In [39]:
acts = np.array([1, 0, 0, 1])
preds = np.array([0.9, 0.1, 0.2, 0.8])
binary_loss(acts, preds)

0.164252033486018

Note that in our toy example above our accuracy is 100% and our loss is 0.16. Compare that to a loss of 0.03 that we are getting while predicting cats and dogs. Exercise: play with `preds` to get a lower loss for this example. 

**Example:** Here is an example on how to compute the loss for one example of binary classification problem. Suppose for an image x with label 1 and your model gives it a prediction of 0.9. For this case the loss should be small because our model is predicting a label $1$ with high probability.

`loss = -log(0.9) = 0.10`

Now suppose x has label 0 but our model is predicting 0.9. In this case our loss should be much larger.

loss = -log(1-0.9) = 2.30

- Exercise: look at the other cases and convince yourself that this make sense.
- Exercise: how would you rewrite `binary_loss` using `if` instead of `*` and `+`?

Why not just maximize accuracy? The binary classification loss is an easier function to optimize.

## Single prediction

In [40]:
trn_tfms,val_tfms = tfms_from_model(arch, sz)
img = val_tfms(Image.open(PATH+img_path))
learn.predict_array(img[None]))
np.argmax(preds)

SyntaxError: invalid syntax (<ipython-input-40-fa9e85b91d86>, line 3)

## Convert Pytorch to ONNX Model

In [14]:
!mkdir -p models/plants-seedlings-classification

In [15]:
MODEL_PATH = "models/plant-seedlings-classification"

In [65]:
#!touch {MODEL_PATH}/model.pth

In [66]:
#torch.save(learn.model, f'{MODEL_PATH}/pth')

In [58]:
!apt-get install convert

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package convert


In [74]:
!pip install utils

You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
pthmodel = torch.load(f'{MODEL_PATH}/pth')

In [18]:
!python ./neural_style.py eval --content-image dummy.jpg --output-image dummy-out.jpg --model {MODEL_PATH}/model.pth --cuda 0 --export_onnx {MODEL_PATH}/model.onnx

Traceback (most recent call last):
  File "./neural_style.py", line 238, in <module>
    main()
  File "./neural_style.py", line 234, in main
    stylize(args)
  File "./neural_style.py", line 134, in stylize
    with torch.no_grad():
AttributeError: module 'torch' has no attribute 'no_grad'
